In [33]:
from statsmodels.tsa.api import AutoReg, VAR
from stargazer.stargazer import Stargazer
from geopy.distance import great_circle
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np

In [34]:
data = pd.read_csv(r"C:\Users\VY72PC\OneDrive - ING\Documents\STARIMA Project\Data\pm25_weer.csv")
data

,id,no2,pm10,pm10_cal,pm10_fac,pm10_max,pm10_min,pm25,pm25_cal,pm25_fac,...,YYYYMMDD,jaar,maand,weeknummer,dag,H,T,U,DD,FH
0,1,NaN,8.195,8.774,1.070000,8.39,8.00,4.000,1.640,0.410000,...,20190712,2019,7,28,12,14,173,86,320,60
1,4,NaN,17.258,18.165,1.052000,20.10,15.60,11.176,4.256,0.381000,...,20190712,2019,7,28,12,14,173,86,320,60
2,5,NaN,19.952,20.497,1.027000,24.40,16.50,10.810,3.996,0.370000,...,20190712,2019,7,28,12,14,173,86,320,60
3,7,NaN,20.278,21.216,1.042000,24.40,16.20,11.510,4.292,0.371000,...,20190712,2019,7,28,12,14,173,86,320,60
4,10,NaN,18.099,19.186,1.062000,21.90,14.70,8.942,3.471,0.390000,...,20190712,2019,7,28,12,14,173,86,320,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349808,199,33.4319,24.660,16.796,0.681103,28.50,20.50,11.910,9.877,0.829303,...,20221022,2022,10,42,22,23,127,92,140,20
1349809,200,43.1974,0.000,0.000,NaN,0.00,0.00,0.000,0.000,NaN,...,20221022,2022,10,42,22,23,127,92,140,20
1349810,212,2.5000,23.074,22.407,0.971261,29.30,18.29,7.887,11.534,1.461850,...,20221022,2022,10,42,22,23,127,92,140,20
1349811,213,39.1156,37.694,36.569,0.970257,60.80,24.40,9.182,13.432,1.463180,...,20221022,2022,10,42,22,23,127,92,140,20


In [35]:
data.columns

Index(['id', 'no2', 'pm10', 'pm10_cal', 'pm10_fac', 'pm10_max', 'pm10_min',
       'pm25', 'pm25_cal', 'pm25_fac', 'pm25_max', 'pm25_min', 'timestamp',
       'components', 'latitude', 'longitude', 'name', 'sensortype', 'tag',
       'datum', 'tijd', 'weekdag', 'uur', '#STN', 'YYYYMMDD', 'jaar', 'maand',
       'weeknummer', 'dag', 'H', 'T', 'U', 'DD', 'FH'],
      dtype='object')

In [36]:
# print(data["T"].unique())
# print(data["U"].unique())
# print(data[data.DD == 990])

In [37]:
data.drop(data.iloc[:, 0:7], axis=1, inplace=True)
data.drop(["jaar", "maand", "weeknummer", "#STN", "timestamp", "components", "dag", "tijd", "T", "datum", "H", "weekdag", "FH", "sensortype"], axis=1, inplace=True)
# data

In [40]:
grouped_df = data.groupby(["YYYYMMDD", "tag"])["pm25", "longitude", "latitude", "DD", "U"].mean().copy().reset_index()

<ipython-input-40-61ebd410a34e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_df = data.groupby(["YYYYMMDD", "tag"])["pm25", "longitude", "latitude", "DD", "U"].mean().copy().reset_index()


In [41]:
grouped_df.head(5)

,YYYYMMDD,tag,pm25,longitude,latitude,DD,U
0,20190625,Amsterdam,16.602565,4.866208,52.359714,170.434783,69.739130
1,20190625,Beverwijk,18.822000,4.650824,52.492369,276.000000,77.400000
2,20190626,Amsterdam,21.974609,4.866208,52.359714,126.521739,81.043478
3,20190626,Beverwijk,2.851417,4.657933,52.485871,19.166667,77.333333
4,20190627,Amsterdam,4.484500,4.866208,52.359714,37.916667,73.458333


In [43]:
Locations = grouped_df["tag"].unique()
LocDict = dict()

for i in range(len(Locations)):
    LocDict[Locations[i]] = (grouped_df[grouped_df.tag == Locations[i]]["latitude"].mean(), grouped_df[grouped_df.tag == Locations[i]]["longitude"].mean())

LocDict

{'Amsterdam': (52.38556547178074, 4.901327224364788),
 'Beverwijk': (52.48636675602225, 4.6585847376827525),
 'Heemskerk': (52.50593616684639, 4.671953092146707),
 'Wijk aan Zee': (52.493443775747366, 4.5982457823463205),
 'Velsen-Noord': (52.47382287197285, 4.647238399148998),
 'Driehuis': (52.44719482753771, 4.636977030738782),
 'IJmuiden': (52.45820717461738, 4.615567828011167),
 'Velsen-Zuid': (52.46234499999992, 4.637255999999981),
 'Zaandam': (52.45848175659799, 4.824912486862416),
 'Koog aan de Zaan': (52.46407900000035, 4.811263999999972),
 'Uithoorn': (52.23809099999961, 4.808228999999986)}

In [44]:
W = np.zeros((11, 11))

for i in range(len(LocDict)):
    for j in range(len(LocDict)):
        if i != j:
            W[i, j] = 1 / great_circle(LocDict[Locations[i]], LocDict[Locations[j]]).km
            
pd.DataFrame(W)

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,0.050226,0.048748,0.042035,0.050438,0.052102,0.047633,0.050410,0.103925,0.093859,0.056891
1,0.050226,0.000000,0.424320,0.240358,0.627957,0.217623,0.233800,0.329302,0.085581,0.094041,0.033993
2,0.048748,0.424320,0.000000,0.193082,0.253583,0.143926,0.152950,0.185648,0.086023,0.095066,0.032065
3,0.042035,0.240358,0.193082,0.000000,0.251844,0.173214,0.244498,0.229791,0.063146,0.067607,0.031474
4,0.050438,0.627957,0.253583,0.251844,0.000000,0.328792,0.362306,0.692351,0.082260,0.089577,0.035210
5,0.052102,0.217623,0.143926,0.173214,0.328792,0.000000,0.526740,0.593567,0.078144,0.083627,0.038463
6,0.047633,0.233800,0.152950,0.244498,0.362306,0.526740,0.000000,0.649445,0.070501,0.075331,0.036030
7,0.050410,0.329302,0.185648,0.229791,0.692351,0.593567,0.649445,0.000000,0.078608,0.084816,0.036354
8,0.103925,0.085581,0.086023,0.063146,0.082260,0.078144,0.070501,0.078608,0.000000,0.897148,0.040762
9,0.093859,0.094041,0.095066,0.067607,0.089577,0.083627,0.075331,0.084816,0.897148,0.000000,0.039794


In [45]:
data2 = grouped_df.copy()
data2["New"] = data2["YYYYMMDD"].astype(str)
del data2["YYYYMMDD"]

In [46]:
data2.set_index("New", inplace=True)

In [47]:
del data2["latitude"]
del data2["longitude"]
data2

,tag,pm25,DD,U
New,,,,
20190625,Amsterdam,16.602565,170.434783,69.739130
20190625,Beverwijk,18.822000,276.000000,77.400000
20190626,Amsterdam,21.974609,126.521739,81.043478
20190626,Beverwijk,2.851417,19.166667,77.333333
20190627,Amsterdam,4.484500,37.916667,73.458333
...,...,...,...,...
20221022,IJmuiden,6.112711,187.169811,84.283019
20221022,Uithoorn,7.120000,186.250000,84.583333
20221022,Velsen-Noord,6.428708,186.250000,84.583333


In [49]:
UniqueNames = data2.tag.unique()

DataFrameDict = {elem : pd.DataFrame() for elem in UniqueNames}

for key in DataFrameDict.keys():
    DataFrameDict[key] = data2[:][data2.tag == key]

DataFrameDict["Amsterdam"]

,tag,pm25,DD,U
New,,,,
20190625,Amsterdam,16.602565,170.434783,69.739130
20190626,Amsterdam,21.974609,126.521739,81.043478
20190627,Amsterdam,4.484500,37.916667,73.458333
20190628,Amsterdam,7.430917,49.583333,71.833333
20190629,Amsterdam,7.474792,117.083333,58.166667
...,...,...,...,...
20221018,Amsterdam,59.648775,176.828194,83.555066
20221019,Amsterdam,49.008481,76.592357,82.245223
20221020,Amsterdam,57.314282,128.169014,93.056338


In [ ]:
# for key in tqdm(DataFrameDict):
#     plt.figure(figsize=(16, 8))
#     plt.plot(DataFrameDict[key].pm25, label = key)
# plt.show

In [ ]:
for key in tqdm(DataFrameDict):
    DataFrameDict[key].rename(columns={"pm25":key}, inplace=True)
    del DataFrameDict[key]["tag"]
    # print(DataFrameDict[key])

In [ ]:
# for key in tqdm(DataFrameDict):
#     model = AutoReg(DataFrameDict[key], lags=1).fit()
#     print(f'{key}\n', model.params)

In [ ]:
df = pd.DataFrame(DataFrameDict["Amsterdam"].copy())
for key in DataFrameDict:
    df = df.combine_first(DataFrameDict[key])

In [ ]:
for column in df:
    mean_value = df[column].mean()
    df[column].fillna(value=mean_value, inplace = True)
    
df

In [ ]:
VARModel = VAR(df).fit()

In [ ]:
WY = pd.DataFrame(np.matmul(df.to_numpy(), W))

i = 0
for key in DataFrameDict:
    WY.rename(columns={i:f'{key}'}, inplace=True)
    i += 1

WY

In [ ]:
SVAR = VAR(WY).fit()
SVAR.summary()